In [1]:
import tensorflow as tf
import os 
import SimpleITK as sitk 
from classification.preprocessing import *

In [2]:
trained_model_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/20210705-15_03_50/trained_model_20210705-15_03_50.h5'
model = tf.keras.models.load_model(trained_model_path)
model.summary()

Model: "classic_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 256, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 1018, 250, 32 1600        input_1[0][0]                    
__________________________________________________________________________________________________
max_pool1 (MaxPooling2D)        (None, 509, 125, 32) 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 507, 123, 64) 18496       max_pool1[0][0]                  
______________________________________________________________________________________

In [3]:
test_directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/test'
files = os.listdir(test_directory)
liste_files = []
for x in files : 
    liste_files.append(os.path.join(test_directory, x))
sorted_test = sorted(liste_files)
X_test = []
y_test = []
for i in range(0, len(sorted_test), 2):
    X_test.append(sitk.GetArrayFromImage(sitk.ReadImage(sorted_test[i+1])))
    y_test.append(get_label_from_json(sorted_test[i]))

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print(X_test.shape)
print(y_test.shape)

(326, 1024, 256, 1)
(326, 4)


In [5]:
#evaluate
result = model.evaluate(X_test,  {'head': y_test[:,0], 
                                    'legs': y_test[:,1],
                                    'right_arm' : y_test[:,2],
                                    'left_arm' : y_test[:,3] ,
                                    })
dict(zip(model.metrics_names, result)) 

11/11 [==============================] - 7s 261ms/step - loss: 0.0278 - head_loss: 0.0508 - legs_loss: 0.0379 - right_arm_loss: 0.0061 - left_arm_loss: 0.0163 - head_head_accuracy: 0.9877 - legs_legs_accuracy: 0.9847 - right_arm_right_arm_accuracy: 1.0000 - left_arm_left_arm_accuracy: 0.9969


{'loss': 0.027774466201663017,
 'head_loss': 0.050832413136959076,
 'legs_loss': 0.037949759513139725,
 'right_arm_loss': 0.006051304750144482,
 'left_arm_loss': 0.01626438833773136,
 'head_head_accuracy': 0.987730085849762,
 'legs_legs_accuracy': 0.9846625924110413,
 'right_arm_right_arm_accuracy': 1.0,
 'left_arm_left_arm_accuracy': 0.9969325065612793}

In [4]:
#predictions
inferences = model.predict(X_test)

In [5]:
len(inferences)
#inference[0] head 
#inferences[1] legs 
#infernece[2] right arm 
#inference[3] left arm 

4

In [6]:
from classification.inference import * 
inferences_decoded = decodage_predictions(inferences)

In [7]:
inferences_decoded[0]

['Vertex', 'Hips', 'up', 'up']

In [8]:
truth_decoded = decodage_truth(y_test)

In [11]:
truth_decoded[0]

['Vertex', 'Hips', 'up', 'up']

In [9]:
#generate image with true/false predictions and save them 
directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/inference'
affichage(X_test, inferences_decoded, truth_decoded, directory)

In [ ]:
#GENERATE MIP FROM NIFTI 

from dicom_to_cnn.model.reader.Nifti import Nifti 
from dicom_to_cnn.model.post_processing.mip.MIP_Generator import MIP_Generator

nifti_path = ''
resampled_array = Nifti(nifti_path).resample(shape_matrix=(256, 256, 1024), shape_physic=(700, 700, 2000))
resampled_array[np.where(resampled_array < 500)] = 0 #500 UH
normalize = resampled_array[:,:,:,]/np.max(resampled_array) #normalize
mip_generator = MIP_Generator(normalize)
mip = mip_generator.project(angle=0)
mip = np.expand_dims(mip, -1)
#mip of size 1024 256 1